In [0]:
import torch
import time
import random
import torch.nn as nn 
import numpy as np 
from torch.autograd import Variable
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_



class MyNet(nn.Module):
    def __init__(self,layers):
        super(MyNet,self).__init__()
        self.relu=nn.ReLU()
        self.hidden=nn.ModuleList()
        for input_size,output_size in zip(layers,layers[1:]):
            self.hidden.append(nn.Linear(input_size,output_size,bias=True))
        for i in range(len(layers)-1):
            kaiming_uniform_(self.hidden[i].weight,nonlinearity='relu')
        xavier_uniform_(self.hidden[-1].weight)
        

    def forward(self,active):
        for i in range(len(self.hidden)-1):
            active=self.hidden[i](active)
            active=self.relu(active)
        active=self.hidden[-1](active)
        active=nn.Softmax(dim=1)(active)
        
        return active
    

class NNs:
    def __init__(self,dim_feature,n_class,batch_size,n_epochs,layers_size,tol,lr=0.01):
        self.dim_feature=dim_feature
        self.n_class=n_class
        self.batch_size=batch_size
        self.n_epochs=n_epochs
        self.lr=lr
        self.layers_size=layers_size
        self.X_train=[]
        self.Y_train=[]
        self.tol=tol
    
    def get_batch(self,i,batch_size):
        batch_x=self.X_train[i*batch_size:i*batch_size+batch_size]
        batch_y=self.Y_train[i*batch_size:i*batch_size+batch_size]
        return batch_x,batch_y
    
    def fit(self,X_train,Y_train):
        rd=random.sample(range(len(Y_train)),len(Y_train))
        self.X_train=np.array(X_train)[rd]
        self.Y_train=np.array(Y_train)[rd]
        self.net=MyNet(self.layers_size)
        self.criterion=nn.CrossEntropyLoss()
        self.optimizer=torch.optim.SGD(self.net.parameters(),self.lr,momentum=0.8)
        total_batch= int(np.ceil(self.X_train.shape[0]/self.batch_size))

        self.net.train()
        for epoch in range(self.n_epochs):          
            for i in range(total_batch):
                # forward pass
                batch_x,batch_y=self.get_batch(i,self.batch_size)
                articles=Variable(torch.FloatTensor(batch_x))
                labels=Variable(torch.LongTensor(batch_y))
                self.optimizer.zero_grad()
                y_pre=self.net(articles)

                #compute loss
                loss=self.criterion(y_pre,labels)

                #backward pass
                loss.backward()
                self.optimizer.step()

                if(i%20==0):
                    print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
 				              %(epoch+1, self.n_epochs, i+1, total_batch, loss.item()))
                if(loss.item()<self.tol):
                    return
    
    def predict(self,X_test,Y_test):
        Y_test=np.array(Y_test)
        articles = Variable(torch.FloatTensor(X_test))
        outputs = self.net(articles)
        pre = torch.max(outputs.data, 1).indices.numpy()
        precision=sum(pre==Y_test)/len(Y_test)
    
        return precision


In [0]:
def loaddata(pathin):
    def convert_dense_vector(text,dim):
        v=np.zeros(dim)
        x_fea=text.split()
        for x in x_fea:
            fea=x.split(':')
            v[int(fea[0])]=float(fea[1])
        return v

    with open(pathin,'r') as f:
        d_lines=f.read().splitlines()
    '''
    with open("/content/drive/My Drive/Project2/Datasets/words_idf",'r') as f:
        dim=f.read().splitlines()
    '''
    dim=20167

    X=[]
    Y=[]
    for doc in d_lines:
        fea_doc=doc.split('<<>>')
        Y.append(int(fea_doc[0]))
        X.append(convert_dense_vector(fea_doc[2],dim))
    
    return X,Y


x_train,y_train=loaddata("/content/drive/My Drive/Project2/Datasets/train_tfidf_df=3")
x_test,y_test=loaddata("/content/drive/My Drive/Project2/Datasets/test_tfidf")

In [17]:
# optimize with SGD , lr=0.01
layers_size=[20167,100,50,20]
t=time.time()
model=NNs(20167,20,100,30,layers_size=layers_size,lr=0.01,tol=2.08)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/30], Step [1/114], Loss: 2.9956
Epoch [1/30], Step [21/114], Loss: 2.9959
Epoch [1/30], Step [41/114], Loss: 2.9964
Epoch [1/30], Step [61/114], Loss: 2.9952
Epoch [1/30], Step [81/114], Loss: 2.9962
Epoch [1/30], Step [101/114], Loss: 2.9954
Epoch [2/30], Step [1/114], Loss: 2.9954
Epoch [2/30], Step [21/114], Loss: 2.9957
Epoch [2/30], Step [41/114], Loss: 2.9962
Epoch [2/30], Step [61/114], Loss: 2.9951
Epoch [2/30], Step [81/114], Loss: 2.9960
Epoch [2/30], Step [101/114], Loss: 2.9953
Epoch [3/30], Step [1/114], Loss: 2.9953
Epoch [3/30], Step [21/114], Loss: 2.9956
Epoch [3/30], Step [41/114], Loss: 2.9960
Epoch [3/30], Step [61/114], Loss: 2.9949
Epoch [3/30], Step [81/114], Loss: 2.9959
Epoch [3/30], Step [101/114], Loss: 2.9951
Epoch [4/30], Step [1/114], Loss: 2.9951
Epoch [4/30], Step [21/114], Loss: 2.9954
Epoch [4/30], Step [41/114], Loss: 2.9958
Epoch [4/30], Step [61/114], Loss: 2.9947
Epoch [4/30], Step [81/114], Loss: 2.9957
Epoch [4/30], Step [101/114], Loss:

In [18]:
# optimize with SGD , lr=0.01
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.1327668613913967
train= 0.15953685699133816


In [15]:
# optimize with SGD , lr=0.5
layers_size=[20167,100,50,20]
t=time.time()
model=NNs(20167,20,100,30,layers_size=layers_size,lr=0.5,tol=2.08)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/30], Step [1/114], Loss: 2.9963
Epoch [1/30], Step [21/114], Loss: 2.9947
Epoch [1/30], Step [41/114], Loss: 2.9929
Epoch [1/30], Step [61/114], Loss: 2.9872
Epoch [1/30], Step [81/114], Loss: 2.9609
Epoch [1/30], Step [101/114], Loss: 2.8637
Epoch [2/30], Step [1/114], Loss: 2.7711
Epoch [2/30], Step [21/114], Loss: 2.7045
Epoch [2/30], Step [41/114], Loss: 2.5253
Epoch [2/30], Step [61/114], Loss: 2.4980
Epoch [2/30], Step [81/114], Loss: 2.4497
Epoch [2/30], Step [101/114], Loss: 2.3561
Epoch [3/30], Step [1/114], Loss: 2.3955
Epoch [3/30], Step [21/114], Loss: 2.3656
Epoch [3/30], Step [41/114], Loss: 2.3636
Epoch [3/30], Step [61/114], Loss: 2.3757
Epoch [3/30], Step [81/114], Loss: 2.3504
Epoch [3/30], Step [101/114], Loss: 2.2647
Epoch [4/30], Step [1/114], Loss: 2.3010
Epoch [4/30], Step [21/114], Loss: 2.2713
Epoch [4/30], Step [41/114], Loss: 2.3076
Epoch [4/30], Step [61/114], Loss: 2.2895
Epoch [4/30], Step [81/114], Loss: 2.2540
Epoch [4/30], Step [101/114], Loss:

In [16]:
# optimize with SGD , lr=0.5
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.7701805629314923
train= 0.9546579459077249


In [8]:
# optimize with Adam, lr=0.001
layers_size=[20167,100,50,20]
t=time.time()
model=NNs(20167,20,100,30,layers_size=layers_size,lr=0.001,tol=2.08)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/30], Step [1/114], Loss: 2.9965
Epoch [1/30], Step [21/114], Loss: 2.9660
Epoch [1/30], Step [41/114], Loss: 2.7705
Epoch [1/30], Step [61/114], Loss: 2.5819
Epoch [1/30], Step [81/114], Loss: 2.4951
Epoch [1/30], Step [101/114], Loss: 2.3079
Epoch [2/30], Step [1/114], Loss: 2.2713
Epoch [2/30], Step [21/114], Loss: 2.3249
Epoch [2/30], Step [41/114], Loss: 2.1862
Epoch [2/30], Step [61/114], Loss: 2.1564
Epoch [2/30], Step [81/114], Loss: 2.1728
Epoch [2/30], Step [101/114], Loss: 2.1365
Epoch [3/30], Step [1/114], Loss: 2.1302
Epoch [3/30], Step [21/114], Loss: 2.1332
Epoch [3/30], Step [41/114], Loss: 2.1116
Epoch [3/30], Step [61/114], Loss: 2.1141
Epoch [3/30], Step [81/114], Loss: 2.0975
Epoch [3/30], Step [101/114], Loss: 2.0932
time train = 12.344376564025879


In [9]:
# optimize with Adam , lr=0.001
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.8368295273499734
train= 0.9880678805020329
